# 🎯 YOLO Custom Training Workflow — Complete Guide

This notebook walks you through the **entire process** of training a custom YOLOv8 model:

1. **Setup** — Create dataset folder structure
2. **Collect** — Capture images from your webcam
3. **Annotate** — Label images using our web-based Annotator tool
4. **Split** — Divide into training and validation sets
5. **Train** — Train YOLOv8 on your custom data
6. **Evaluate** — Check how well the model performs
7. **Predict** — Test on new images or webcam
8. **Export** — Convert to other formats for deployment

**No Roboflow, no cloud, no internet needed** (after initial setup).

---

## Step 0: Prerequisites

Make sure you have:
- `ultralytics` installed (`uv sync` from the project root)
- A webcam (optional, for capturing training images)
- At least 50 images of the object(s) you want to detect

In [ ]:
# Verify ultralytics is installed
import ultralytics
print(f"Ultralytics version: {ultralytics.__version__}")
ultralytics.checks()

In [ ]:
# Import everything we need
from ultralytics import YOLO
import cv2
import os
import glob
import shutil
import random
from pathlib import Path
from IPython.display import Image, display, clear_output
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Set up paths
# Change DATASET_NAME to whatever you're detecting
DATASET_NAME = "my_dataset"
BASE_DIR = Path(".").resolve()  # YoloExamples/ directory
DATASET_DIR = BASE_DIR / DATASET_NAME

print(f"Working directory: {BASE_DIR}")
print(f"Dataset directory: {DATASET_DIR}")

---
## Step 1: Define Your Classes

What objects do you want to detect? List them here.

**Examples:**
- Safety equipment: `["helmet", "no_helmet"]`
- Fruits: `["apple", "banana", "orange"]`
- Custom parts: `["screw", "bolt", "washer"]`
- Pets: `["my_cat", "my_dog"]`

In [ ]:
# ════════════════════════════════════════════════════════════
#  CHANGE THIS to your object classes!
# ════════════════════════════════════════════════════════════
CLASSES = ["helmet", "no_helmet"]

print(f"Number of classes: {len(CLASSES)}")
for i, cls in enumerate(CLASSES):
    print(f"  Class {i}: {cls}")

---
## Step 2: Create Dataset Folder Structure

YOLO expects this exact structure:
```
my_dataset/
├── data.yaml          <- Config file (auto-generated below)
├── train/
│   ├── images/        <- 80% of your images
│   └── labels/        <- Annotation .txt files
└── val/
    ├── images/        <- 20% of your images
    └── labels/        <- Annotation .txt files
```

In [ ]:
# Create all directories
for split in ["train", "val"]:
    for folder in ["images", "labels"]:
        path = DATASET_DIR / split / folder
        path.mkdir(parents=True, exist_ok=True)
        print(f"  Created: {path}")

# Create data.yaml configuration file
data_yaml_path = DATASET_DIR / "data.yaml"
data_yaml_content = f"""# YOLOv8 Custom Dataset Configuration
# Auto-generated by yolo_training_workflow.ipynb

train: train/images
val: val/images

nc: {len(CLASSES)}
names: {CLASSES}
"""

with open(data_yaml_path, "w") as f:
    f.write(data_yaml_content)

print(f"\nCreated: {data_yaml_path}")
print(f"\ndata.yaml contents:")
print(data_yaml_content)

---
## Step 3: Capture Training Images (Optional)

If you already have images, skip this step and copy them into
`my_dataset/train/images/` and `my_dataset/val/images/`.

Otherwise, use the cell below to capture images from your webcam.

**Tips for good training images:**
- Capture from **different angles** and **distances**
- Include **different lighting** conditions
- Include **different backgrounds**
- Aim for **50-500 images** per class
- More variety = better model

In [ ]:
def capture_images(save_dir, num_images=50, delay_ms=500):
    """
    Capture images from webcam for training data.
    
    Args:
        save_dir: Directory to save captured images
        num_images: Number of images to capture
        delay_ms: Delay between captures in milliseconds
    
    Controls:
        SPACE  - Capture an image manually
        a      - Toggle auto-capture mode
        q      - Quit capturing
    """
    os.makedirs(save_dir, exist_ok=True)
    
    # Count existing images to continue numbering
    existing = glob.glob(os.path.join(save_dir, "*.jpg"))
    count = len(existing)
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("ERROR: Could not open webcam")
        return
    
    auto_capture = False
    print(f"Webcam opened. Saving to: {save_dir}")
    print(f"Already have {count} images.")
    print("SPACE=capture, a=auto-capture, q=quit")
    
    while count < num_images:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Show info on frame
        info = f"Captured: {count}/{num_images}"
        mode = " [AUTO]" if auto_capture else " [MANUAL]"
        cv2.putText(frame, info + mode, (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                    (0, 255, 0), 2)
        cv2.imshow("Capture Training Images", frame)
        
        key = cv2.waitKey(delay_ms if auto_capture else 30)
        key = key & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord(' ') or auto_capture:
            # Save the frame
            filename = f"img_{count:04d}.jpg"
            filepath = os.path.join(save_dir, filename)
            cv2.imwrite(filepath, frame)
            count += 1
            if not auto_capture:
                print(f"  Saved: {filename}")
        elif key == ord('a'):
            auto_capture = not auto_capture
            state = "ON" if auto_capture else "OFF"
            print(f"  Auto-capture: {state}")
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"\nDone! Captured {count} images in {save_dir}")


# ════════════════════════════════════════════════════════════
#  Uncomment the line below to start capturing!
# ════════════════════════════════════════════════════════════
# capture_images(str(DATASET_DIR / "all_images"), num_images=100)

---
## Step 4: Annotate Images

Now we need to draw bounding boxes around objects in each image.

Use our **web-based Annotator tool** — no internet or external services needed!

### How to annotate:

1. Open a **separate terminal** and run:
```bash
uv run python YoloExamples/annotate/app.py
```

2. Open **http://localhost:5000** in your browser

3. Click **"Select Folder"** and navigate to your `train/images/` folder

4. Add your classes (e.g., `helmet`, `no_helmet`)

5. Draw bounding boxes on each image:
   - Select a class → click & drag to draw a box
   - Switch class → draw boxes for different objects
   - **Multiple objects per image**: each box becomes a line in the `.txt` label file

6. Repeat for `val/images/`

### Multi-class labeling example:

If an image has a person WITH a helmet and another WITHOUT:
```
0 0.4807 0.2089 0.3461 0.3750    ← class 0 (helmet)
1 0.7200 0.5100 0.2000 0.3000    ← class 1 (no_helmet)
```
Each line = one bounding box. Different lines can have different class IDs.

### Keyboard shortcuts (in the Annotator):
| Key | Action |
|-----|--------|
| `1-9` | Select class |
| `D` / `→` | Next image (auto-saves) |
| `A` / `←` | Previous image |
| `S` | Save labels |
| `Z` | Undo last box |

In [ ]:
# ════════════════════════════════════════════════════════════
#  Annotation is done via the web-based Annotator tool.
#  Run in a separate terminal:
#    uv run python YoloExamples/annotate/app.py
#  Then open http://localhost:5000 in your browser.
#
#  Point it to your train/images/ and val/images/ folders
#  to draw bounding boxes and save YOLO-format labels.
# ════════════════════════════════════════════════════════════
print("Annotate your images using the web Annotator tool:")
print("  uv run python YoloExamples/annotate/app.py")
print("  Then open http://localhost:5000")
print()
print("Once done, come back here and continue with Step 5.")

---
## Step 5: Auto-Split Images (Optional)

If you have all your images in one folder (e.g., `all_images/`),
this cell will automatically split them 80/20 into train/val.

In [ ]:
def auto_split_dataset(source_dir, dataset_dir, ratio=0.8):
    """
    Split images + labels from source_dir into train/val.
    
    Args:
        source_dir: Folder with images (and optionally labels)
        dataset_dir: Target dataset directory
        ratio: Fraction for training (0.8 = 80% train, 20% val)
    """
    exts = (".jpg", ".jpeg", ".png", ".bmp", ".webp")
    images = [
        f for f in os.listdir(source_dir)
        if os.path.splitext(f)[1].lower() in exts
    ]
    
    if not images:
        print(f"No images found in {source_dir}")
        return
    
    random.shuffle(images)
    split = int(len(images) * ratio)
    train_imgs = images[:split]
    val_imgs = images[split:]
    
    print(f"Total: {len(images)} | "
          f"Train: {len(train_imgs)} | Val: {len(val_imgs)}")
    
    for split_name, img_list in [("train", train_imgs),
                                  ("val", val_imgs)]:
        img_dir = os.path.join(dataset_dir, split_name, "images")
        lbl_dir = os.path.join(dataset_dir, split_name, "labels")
        os.makedirs(img_dir, exist_ok=True)
        os.makedirs(lbl_dir, exist_ok=True)
        
        for img_name in img_list:
            # Copy image
            src = os.path.join(source_dir, img_name)
            shutil.copy2(src, img_dir)
            
            # Copy label if exists
            base = os.path.splitext(img_name)[0]
            lbl_src = os.path.join(source_dir, base + ".txt")
            if os.path.exists(lbl_src):
                shutil.copy2(lbl_src, lbl_dir)
    
    print("Split complete!")


# ════════════════════════════════════════════════════════════
#  Uncomment to auto-split from an 'all_images' folder:
# ════════════════════════════════════════════════════════════
# auto_split_dataset(
#     str(DATASET_DIR / "all_images"),
#     str(DATASET_DIR),
#     ratio=0.8
# )

---
## Step 6: Verify Dataset

Before training, let's check that everything looks correct.

In [ ]:
# Count images and labels
for split in ["train", "val"]:
    img_dir = DATASET_DIR / split / "images"
    lbl_dir = DATASET_DIR / split / "labels"
    
    imgs = list(img_dir.glob("*")) if img_dir.exists() else []
    lbls = list(lbl_dir.glob("*.txt")) if lbl_dir.exists() else []
    
    # Filter to only image files
    img_exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
    imgs = [i for i in imgs if i.suffix.lower() in img_exts]
    
    print(f"{split:>5}: {len(imgs)} images, {len(lbls)} labels")
    
    if len(imgs) != len(lbls):
        print(f"  ⚠ WARNING: Image/label count mismatch!")
        # Find unlabeled images
        labeled = {l.stem for l in lbls}
        unlabeled = [i.name for i in imgs if i.stem not in labeled]
        if unlabeled:
            print(f"  Unlabeled: {unlabeled[:5]}...")

print(f"\ndata.yaml: {data_yaml_path}")
if data_yaml_path.exists():
    print(data_yaml_path.read_text())

In [ ]:
# Visualize a few annotated images to verify labels are correct
def show_annotated_samples(image_dir, label_dir, classes,
                           num_samples=4):
    """Display annotated images with bounding boxes."""
    img_exts = (".jpg", ".jpeg", ".png", ".bmp")
    images = [
        f for f in os.listdir(image_dir)
        if os.path.splitext(f)[1].lower() in img_exts
    ]
    
    if not images:
        print("No images found.")
        return
    
    samples = images[:num_samples]
    fig, axes = plt.subplots(1, len(samples),
                             figsize=(5 * len(samples), 5))
    if len(samples) == 1:
        axes = [axes]
    
    colors_rgb = [
        (0, 1, 0), (1, 0, 0), (0, 0, 1), (1, 1, 0),
        (0, 1, 1), (1, 0, 1)
    ]
    
    for ax, img_name in zip(axes, samples):
        img_path = os.path.join(image_dir, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]
        
        ax.imshow(img)
        ax.set_title(img_name, fontsize=8)
        ax.axis('off')
        
        # Draw labels
        base = os.path.splitext(img_name)[0]
        lbl_path = os.path.join(label_dir, base + ".txt")
        if os.path.exists(lbl_path):
            with open(lbl_path) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    cid = int(parts[0])
                    cx = float(parts[1]) * w
                    cy = float(parts[2]) * h
                    bw = float(parts[3]) * w
                    bh = float(parts[4]) * h
                    x1 = cx - bw / 2
                    y1 = cy - bh / 2
                    
                    color = colors_rgb[cid % len(colors_rgb)]
                    name = classes[cid] if cid < len(classes) else str(cid)
                    
                    rect = plt.Rectangle(
                        (x1, y1), bw, bh,
                        linewidth=2, edgecolor=color,
                        facecolor='none'
                    )
                    ax.add_patch(rect)
                    ax.text(x1, y1 - 5, name, color=color,
                            fontsize=8, weight='bold')
    
    plt.tight_layout()
    plt.show()


# ════════════════════════════════════════════════════════════
#  Uncomment to visualize training samples:
# ════════════════════════════════════════════════════════════
# show_annotated_samples(
#     str(DATASET_DIR / "train" / "images"),
#     str(DATASET_DIR / "train" / "labels"),
#     CLASSES
# )

---
## Step 7: Train the Model!

This is where the magic happens. We start from the pre-trained
`yolov8n.pt` (which already knows general features like edges and
shapes) and fine-tune it on YOUR specific objects.

**Training time depends on:**
- Number of images (more = longer)
- Number of epochs (more = longer but potentially better)
- GPU vs CPU (GPU is 10-50x faster)
- Model size (nano is fastest)

**Rough estimates for 100 images, 50 epochs:**
- GPU (NVIDIA): ~5-15 minutes
- CPU (laptop): ~30-90 minutes

In [ ]:
# ════════════════════════════════════════════════════════════
#  Training Configuration — Adjust these!
# ════════════════════════════════════════════════════════════
EPOCHS = 50          # Training rounds (50-100 for small datasets)
IMAGE_SIZE = 640     # Input image size
BATCH_SIZE = 16      # Reduce to 8 or 4 if you get memory errors
BASE_MODEL = "yolov8n.pt"  # Nano model (fastest)

print(f"Model:      {BASE_MODEL}")
print(f"Dataset:    {data_yaml_path}")
print(f"Epochs:     {EPOCHS}")
print(f"Image size: {IMAGE_SIZE}")
print(f"Batch size: {BATCH_SIZE}")

In [ ]:
# Load the pre-trained model
model = YOLO(BASE_MODEL)

# Start training!
# This will print progress for each epoch.
results = model.train(
    data=str(data_yaml_path),
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    name="custom_train",
    project=str(BASE_DIR / "runs" / "detect"),
    exist_ok=True,
    # Data augmentation (helps small datasets generalize)
    hsv_h=0.015,     # Hue variation
    hsv_s=0.7,       # Saturation variation
    hsv_v=0.4,       # Brightness variation
    fliplr=0.5,      # Horizontal flip 50% of the time
    mosaic=1.0,      # Mosaic augmentation (combines 4 images)
    verbose=True,
)

print("\n" + "=" * 60)
print("  Training Complete!")
print("=" * 60)

---
## Step 8: Evaluate Training Results

Let's see how well the model learned.

In [ ]:
# Path to the best trained model
BEST_MODEL = BASE_DIR / "runs" / "detect" / "custom_train" / "weights" / "best.pt"

if BEST_MODEL.exists():
    print(f"Best model: {BEST_MODEL}")
    print(f"Model size: {BEST_MODEL.stat().st_size / 1e6:.1f} MB")
else:
    print("No trained model found. Run training first!")
    # Fall back to base model for demo
    BEST_MODEL = Path(BASE_MODEL)

In [ ]:
# Show training curves
results_img = (BASE_DIR / "runs" / "detect" / "custom_train" 
               / "results.png")

if results_img.exists():
    img = mpimg.imread(str(results_img))
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('Training Results', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No results.png found. Train the model first.")

In [ ]:
# Show confusion matrix
conf_matrix = (BASE_DIR / "runs" / "detect" / "custom_train" 
               / "confusion_matrix.png")

if conf_matrix.exists():
    img = mpimg.imread(str(conf_matrix))
    fig, ax = plt.subplots(1, 1, figsize=(8, 8))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title('Confusion Matrix', fontsize=14)
    plt.tight_layout()
    plt.show()
else:
    print("No confusion matrix found.")

In [ ]:
# Run validation to get metrics
if BEST_MODEL.exists() and data_yaml_path.exists():
    val_model = YOLO(str(BEST_MODEL))
    metrics = val_model.val(data=str(data_yaml_path))
    
    print("\n" + "=" * 40)
    print("  Validation Metrics")
    print("=" * 40)
    print(f"  mAP50:     {metrics.box.map50:.4f}")
    print(f"  mAP50-95:  {metrics.box.map:.4f}")
    print(f"  Precision: {metrics.box.mp:.4f}")
    print(f"  Recall:    {metrics.box.mr:.4f}")
    print()
    print("  What do these mean?")
    print("  - mAP50: How well the model detects objects")
    print("           (higher = better, max = 1.0)")
    print("  - Precision: Of all detections, how many are correct")
    print("  - Recall: Of all real objects, how many were found")
else:
    print("Train a model first to see validation metrics.")

---
## Step 9: Test on New Images

Let's see the trained model in action!

In [ ]:
# Test on a single image
def predict_and_show(model_path, image_path):
    """Run prediction and display the result."""
    model = YOLO(str(model_path))
    results = model(image_path, conf=0.5)
    
    # Get the annotated image
    annotated = results[0].plot()  # BGR numpy array
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    # Display
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    ax.imshow(annotated_rgb)
    ax.axis('off')
    ax.set_title(f'Detections: {len(results[0].boxes)}', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Print detections
    for box in results[0].boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        name = results[0].names[cls_id]
        print(f"  {name}: {conf:.2f}")


# ════════════════════════════════════════════════════════════
#  Uncomment and set the image path to test:
# ════════════════════════════════════════════════════════════
# predict_and_show(BEST_MODEL, "path/to/test_image.jpg")

In [ ]:
# Test on webcam (opens an OpenCV window)
def predict_webcam(model_path):
    """Run the trained model on webcam feed."""
    model = YOLO(str(model_path))
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Could not open webcam")
        return
    
    print("Webcam running. Press 'q' to quit.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame, conf=0.5, verbose=False)
        annotated = results[0].plot()
        
        cv2.imshow("Custom Model - Webcam", annotated)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


# ════════════════════════════════════════════════════════════
#  Uncomment to test on webcam:
# ════════════════════════════════════════════════════════════
# predict_webcam(BEST_MODEL)

---
## Step 10: Export for Deployment

Export your trained model to different formats for deployment
on various platforms.

In [ ]:
# Export to ONNX (universal, works on any platform)
if BEST_MODEL.exists():
    export_model = YOLO(str(BEST_MODEL))
    
    # ONNX — Universal format
    # export_model.export(format="onnx")
    
    # TensorRT — NVIDIA GPUs (fastest)
    # export_model.export(format="engine")
    
    # TFLite — Mobile / Raspberry Pi
    # export_model.export(format="tflite")
    
    # OpenVINO — Intel devices
    # export_model.export(format="openvino")
    
    print("Uncomment the export format you need above.")
    print("Available formats: onnx, engine, tflite, coreml, openvino")
else:
    print("Train a model first!")

---
## Summary: The Complete Workflow

```
1. Define classes          → CLASSES = ["helmet", "no_helmet"]
2. Create folder structure → my_dataset/train/images, etc.
3. Capture/collect images  → Webcam or existing photos
4. Annotate locally        → uv run python YoloExamples/annotate/app.py
5. Split train/val         → 80/20 split
6. Train                   → model.train(data=..., epochs=50)
7. Evaluate                → Check mAP, precision, recall
8. Test                    → Run on new images or webcam
9. Export                  → ONNX, TensorRT, TFLite, etc.
```

### Tips for Better Results

- **More images = better model** (aim for 100+ per class)
- **Diverse images** — vary angle, lighting, background, distance
- **Accurate labels** — take time to draw tight bounding boxes
- **More epochs** — try 100 if 50 isn't enough (watch for overfitting)
- **Larger model** — try `yolov8s.pt` if nano isn't accurate enough
- **Data augmentation** — already enabled by default in our config

### Multi-Class Labeling

Each image's `.txt` label file can have **multiple lines** with **different class IDs**:
```
0 0.4807 0.2089 0.3461 0.3750    ← class 0 (helmet)
1 0.7200 0.5100 0.2000 0.3000    ← class 1 (no_helmet)
```
The Annotator tool handles this automatically — just switch classes and draw boxes.

### Connecting to ESP8266

Once your custom model works, you can use it to control hardware!
See `object_detection.py` and the workshop docs for how to send
UDP commands to the ESP8266 based on detections.